In [ ]:
pip install progressbar

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp

from sklearn.linear_model import LogisticRegression

import string
from string import digits 
import unicodedata
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk import word_tokenize
from sklearn.feature_extraction import text
from progressbar import ProgressBar
import emoji
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [ ]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
X_train = train['comment_text']
y_train = train[classes]

X_test = test['comment_text']

In [ ]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
def preprocessing(_text):
  new = []
  pbar = ProgressBar()
  for t in pbar(_text.values):

    #demojify
    t = emoji.demojize(t)

    #remove number
    t = ''.join([i for i in t if not i.isdigit()])

    #remove stopword
    stopwords = set(text.ENGLISH_STOP_WORDS)
    tokens = tokenize(t)
    text_modified = ''
    for i in tokens:
      if not i in stopwords:
        lemmatizer = WordNetLemmatizer()
        i = lemmatizer.lemmatize(i,'a')
        text_modified += i + ' '

    new.append(t)
  return new

In [ ]:
X_train_f = preprocessing(X_train)
X_test_f = preprocessing(X_test)

In [ ]:
word_vectorizer = TfidfVectorizer(ngram_range=(1,2),tokenizer=tokenize,token_pattern = None,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)

word_vectorizer.fit(X_train_f)
X_train_word = word_vectorizer.transform(X_train_f)
X_test_word = word_vectorizer.transform(X_test_f)

In [ ]:
char_vectorizer = TfidfVectorizer(ngram_range=(2,4),analyzer = 'char',
               strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1, max_features=50000)

char_vectorizer.fit(X_train_f)
X_train_char = char_vectorizer.transform(X_train_f)
X_test_char = char_vectorizer.transform(X_test_f)

In [ ]:
X_train_char_word = sp.sparse.csr_matrix(sp.sparse.hstack([X_train_char, X_train_word]))
X_test_char_word = sp.sparse.csr_matrix(sp.sparse.hstack([X_test_char,X_test_word]))

In [ ]:
subm = pd.DataFrame({'id':test['id']})

In [ ]:
x = X_train_char_word
test_x = X_test_char_word

def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [ ]:
for _class in classes:
    y_train_1 = y_train[_class]
    
    model = LogisticRegression(max_iter=200000,class_weight='balanced')
    
    r = np.log(pr(1,y_train_1.values) / pr(0,y_train_1.values))
    x_nb = x.multiply(r)
    model = LogisticRegression(max_iter=200000,class_weight='balanced')
    print('fitting '+ _class + '...')
    model.fit(x_nb, y_train_1)
    pred = model.predict_proba(test_x.multiply(r))
    subm[_class]=pred[:,1]

In [ ]:
subm.to_csv('submission.csv',index=False)

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')